# Python and Unicode

A quick recap of vocabulary:
- Unicode: A universal mapping between characters and numerical values (usually in hexadecimal). 
- UTF-8: A variable-length encoding implementation, and most commonly used today
- Code unit: The basic component used by an encoding system. UTF-8 code units are byte-sized.
- Code point: The number (usually in hexadecimal) a character is assigned in Unicode. Often prepended with `U+`. UTF-8 encodes code points with one to four code units. 
- Grapheme cluster: What we think of as a single "character" or unit on the screen. It can be composed of multiple code points (more on that below).
- Encoding: the process of assigning numbers to characters
- Decoding: finding a character given a numerical value

You'll note that "character" can be a bit ambiguous in its definition (is it a code point or grapheme cluster), so we'll try and stick with the more precise terms when necessary.

#### A small encoding example

In order to encode a character in UTF-8, we first need to find the character's code point and then encode it according to the table below:

| First code point | Last code point | Byte 1 | Byte 2 | Byte 3 | Byte 4 | Code points | 
| --------: | ---------: | ---------- | :--------: | :--------: | :--------: | ------: | 
| `U+0000`  | `U+007F`   | `0xxxxxxx` | `NULL`     | `NULL`     | `NULL    ` |     128 |
| `U+0080`  | `U+07FF`   | `110xxxxx` | `10xxxxxx` | `NULL`     | `NULL    ` |    1920 |
| `U+0800`  | `U+FFFF`   | `1110xxxx` | `10xxxxxx` | `10xxxxxx` | `NULL    ` |   61440 | 
| `U+10000` | `U+10FFFF` | `11110xxx` | `10xxxxxx` | `10xxxxxx` | `10xxxxxx` | 1048576 |

For example, the 🌵 emoji has code point `U+1F335`. 
1. That range falls into the fourth row, so we know our emoji needs four bytes.
2. We convert `1F335` to binary form: `0001_1111_0011_0011_0101`. 
3. We split in into chunks of 3, 6, 6, and 6 (the number of `x`'s in each byte), dropping the leading zeroes: `000`, `011111`, `001100`, `110101`.
4. We fill in the `x`'s to get our code units / bytes: `1111000 10011111 10001100 10110101`.
5. Convert back to hexadecimal, since bytes are usually referred to in hex: `f0 9f 8c b5`.

We can confirm our work by decoding the bytes into UTF-8:

In [ ]:
bytes.fromhex('f0 9f 8c b5').decode('utf-8')

## Python and Unicode

First, we need some function to help us navigate our way around. Feel free to just execute the cell without reading it, there's some low-level [bit twiddling](https://en.wikipedia.org/wiki/Bit_manipulation#Terminology) that isn't of much importance. If you're curious about what these functions do in greater detail, the appendix at the bottom will explain.

In [ ]:
import unicodedata

def print_bytes(s):
    """Prints the underlying UTF-8 encoding in both hex and binary
    """
    bs = bytearray(s, 'utf-8')
    for i, b in enumerate(bs): 
        print(f'Byte {i+1:02}:\tHex: {b:02x}; Binary: {b:08b}')

def split_units_by_points(bs):
    """Takes in a string or bytearray of UTF-8 code units and 
    attempts to return a list of bytearrays corresponding to code points."""
    if isinstance(bs, str):
        bs = bytearray(bs, 'utf-8')
    code_points = []

    i = 0
    while i < len(bs):
        b = bs[i]
        if b >> 7 == 0b0:
            code_points.append(bs[i:i+1])
            i += 1
        elif b >> 5 == 0b110:     # length 2
            code_points.append(bs[i:i+2])
            i += 2
        elif b >> 4 == 0b1110:    # length 3
            code_points.append(bs[i:i+3])
            i += 3
        elif b >> 3 == 0b11110:   # length 4
            code_points.append(bs[i:i+4])
            i += 4
    return code_points

def codepoints(bs):
    """Takes in a string or bytearray of UTF-8 code units and 
    attempts to return the corresponding Unicode code points.

    This is the same as map(ord, s), mostly here for educational purposes
    
    """
    code_points = []
    chunked_bs = split_units_by_points(bs)
    for bs in chunked_bs:
        if len(bs) == 1:
            code_points.append(bs[0])
        elif len(bs) == 2:
            code_points.append(((0b00011111 & bs[0]) << 6) + \
                                (0b00111111 & bs[1]))
        elif len(bs) == 3:
            code_points.append(((0b00001111 & bs[0]) << 12) + \
                               ((0b00111111 & bs[1]) << 6) + \
                                (0b00111111 & bs[2]))
        elif len(bs) == 4:
            code_points.append(((0b00000111 & bs[0]) << 18) + \
                               ((0b00111111 & bs[1]) << 12) + \
                               ((0b00111111 & bs[2]) << 6) + \
                                (0b00111111 & bs[3]))
        else:
            raise UnicodeDecodeError()
    return code_points

def codepoints_hex(s):
    return list(map(hex, map(ord, s)))

Python, like most modern applications, defaults to Unicode. As a fun side effect, this means that your variable names can be non-English, e.g.

In [ ]:
다섯 = 5
五 = 5
ᎯᏍᎩ = 5
አምስት = 5

다섯 + 五 + ᎯᏍᎩ + አምስት

The full list of valid code points can be found [here](https://www.unicode.org/Public/14.0.0/ucd/DerivedCoreProperties.txt). In string literals, you can refer to any character by its Unicode escape sequence (name or code point) as well:

In [ ]:
cow = '🐮'
cat = '\N{WEARY CAT FACE}'
rabbit = '\u5154'
dog = '\U0001F436'  # emojis neeed a capital U long-form escape sequence since they're more than 16 bits
print(cow, cat, rabbit, dog)

## Potential issues

Thankfully, the world has more or less settled on UTF-8 being the standard character encoding. However, it doesn't have 100% adoption (yet) and the encoding can get fairly complicated, both issues leading to some problems. A few of the more common ones mentioned below:

### 1. Unexpected encodings

This is probably the most common issue you'll run into.

Unfortunately, many (especially older) text datasets still use non-UTF-8 encodings. For example, the [Supreme Court Database](http://scdb.wustl.edu/) uses ISO 8859-1. This is the most common issue you will run into. Libraries such as `pandas` will default to UTF-8, and will likely complain when running into invalid code sequences. Hopefully, the dataset's documentation will provide guidance on which encoding to specify if this is the case. If not, finding the correct one requires a bit of trial and error. 

### 2. `len()` may produce unexpected result

Before you run the following cell, try and predict what the result should be.


In [ ]:
emoji = '🤦🏼‍♂️'
len(emoji)

In order to figure out why this is the case, looking at the internal representation may help:

In [ ]:
codepoints_hex(emoji)

So we have five code points, which is where `len` is getting its result from. If you are curious, the five code points are:
- U+1F926 FACE PALM: The default face palm emoji 🤦.
- U+1F3FC EMOJI MODIFIER FITZPATRICK TYPE-3: The [Fitzpatrick scale](https://en.wikipedia.org/wiki/Fitzpatrick_scale) is a skin color classification system
- U+200D ZERO WIDTH JOINER: The [ZWJ](https://en.wikipedia.org/wiki/Zero-width_joiner) is a control character 
                            telling the computer to join the code points preceding and following
- U+2642 MALE SIGN: The ♂ emoji. Using the ZWJ, this indicates that the face palm emoji is male.
- U+FE0F VARIATION SELECTOR-16: This is another control character, telling the computer that the emoji is to be in color, rather than monochrome.



### 3. Graphemes have multiple possible representations

Due to historical reasons, Unicode has multiple ways to represent the same grapheme, for example, on almost all systems:

```
U+0041 ( A ) LATIN CAPITAL LETTER A
U+0300 ( ̀ ) COMBINING GRAVE ACCENT
```

will look exactly the same as 

```
U+00C0 ( À ) LATIN CAPITAL LETTER A WITH GRAVE
```

This usually is not an issue, since a vast majority of text data is in the composed form (the latter), but problems can still arise. For example, older Macs will actually save filenames in their decomposed form.


### 3.1. Sorting strings may produce unexpected results

[Taken from [Eevee's blog](https://eev.ee/blog/2015/09/12/dark-corners-of-unicode/#everything-you-know-about-text-is-wrong)]

What do you think the following should evaluate as?


In [ ]:
words = ['cafeteria', 'caffeine', 'café']
words.sort()
words

Python sorts by Unicode code point, so "é" (U+00E9) is greater than the English letter "f" (U+0066).


### 3.2. Strings that look exactly the same can have different underlying values

What do you expect the following line to evaluate as?

In [ ]:
e1, e2 = 'è', 'è'
e1 == e2

Can you guess why this might be the case? 

Looking at the internal byte representation may help:

In [ ]:
print_bytes(e1)
print('Code point(s):', [hex(b) for b in codepoints(e1)])
print([unicodedata.name(c) for c in e1])

print('\n---\n')

print_bytes(e2)
print('Code point(s):', [hex(b) for b in codepoints(e2)])
print([unicodedata.name(c) for c in e2])

You can handle this by having Python choose a canonical representation for two similar characters, known as _normalization_, with `unicodedata.normalize`

In [ ]:
unicodedata.normalize('NFC', 'è') == unicodedata.normalize('NFC','è')

# Appendix: Code explainer

This assumes some knowledge of loops and lists, but otherwise, I've tried to make it as accessible as possible. As always, feel free to ask questions on the Ed board, office hours, or in email.

In [ ]:
import unicodedata

[`unicodedata`](https://docs.python.org/3/library/unicodedata.html) is a library with some handy Unicode-related functions

In [ ]:
def print_bytes(s):
    """Prints the underlying UTF-8 encoding in both hex and binary
    """
    bs = bytearray(s, 'utf-8')
    for i, b in enumerate(bs): 
        print(f'Byte {i+1:02}:\tHex: {b:02x}; Binary: {b:08b}')

The line 
```python
bs = bytearray(s, 'utf-8')
```
gets the UTF-8 encoding of the string `s` and stores it in the variable `bs`.

We then iterate through the bytes ([`enumerate()`](https://docs.python.org/3/library/functions.html#enumerate) keeps track of how many iterations have occurred), and print out a line using a [formatted string literals](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals).

Anything in the curly braces `{...}` is evaluated as an expression, rather than treated as a string. Using `{b:02x}` as an example:
1. We split the expression on the colon (`:`) and have `b` on the left and `02x` on the right.
2. `b` is referring to whichever byte value in `bs` we're dealing with in that specific loop iteration.
3. `x` means we want to print the number in hexadecimal, rather than decimal
4. `02` means that we want the length of the string to be at least 2, padding with 0 as necessary. e.g., `'a'` would be turned into `'0a'`.

In [ ]:
def split_units_by_points(bs):
    """Takes in a string or bytearray of UTF-8 code units and 
    attempts to return a list of bytearrays corresponding to code points."""
    if isinstance(bs, str):
        bs = bytearray(bs, 'utf-8')
    code_points = []

    i = 0
    while i < len(bs):
        b = bs[i]
        if b >> 7 == 0b0:
            code_points.append(bs[i:i+1])
            i += 1
        elif b >> 5 == 0b110:     # length 2
            code_points.append(bs[i:i+2])
            i += 2
        elif b >> 4 == 0b1110:    # length 3
            code_points.append(bs[i:i+3])
            i += 3
        elif b >> 3 == 0b11110:   # length 4
            code_points.append(bs[i:i+4])
            i += 4
    return code_points

The first few bits in a code unit let us know if
- how many bytes are used in encoding the code point
- if the byte is a continution byte or a initial byte

Recall the table

| First code point | Last code point | Byte 1 | Byte 2 | Byte 3 | Byte 4 | Code points | 
| --------: | ---------: | ---------- | :--------: | :--------: | :--------: | ------: | 
| `U+0000`  | `U+007F`   | `0xxxxxxx` | `NULL`     | `NULL`     | `NULL    ` |     128 |
| `U+0080`  | `U+07FF`   | `110xxxxx` | `10xxxxxx` | `NULL`     | `NULL    ` |    1920 |
| `U+0800`  | `U+FFFF`   | `1110xxxx` | `10xxxxxx` | `10xxxxxx` | `NULL    ` |   61440 | 
| `U+10000` | `U+10FFFF` | `11110xxx` | `10xxxxxx` | `10xxxxxx` | `10xxxxxx` | 1048576 |

Reading this, we know if a byte starts with `0`, `110`, `1110`, `11110`, we expect the next 1-4 bytes to encode a single code point, respectively. In order to get the first 1-5 bits of a byte, a bit-shift operator (`>>`) is used. `b >> n` is the same as `b // 2 ** n`, or just removing the `n`-rightmost digits from `b` in binary (If you are curious as to why that is identical, consider what `d // 10^a` looks like in decimal).

In [ ]:
def codepoints(bs):
    """Takes in a string or bytearray of UTF-8 code units and 
    attempts to return the corresponding Unicode code points.

    This is the same as map(ord, s), mostly here for educational purposes
    
    """
    code_points = []
    chunked_bs = split_units_by_points(bs)
    for bs in chunked_bs:
        if len(bs) == 1:
            code_points.append(bs[0])
        elif len(bs) == 2:
            code_points.append(((0b00011111 & bs[0]) << 6) + \
                                (0b00111111 & bs[1]))
        elif len(bs) == 3:
            code_points.append(((0b00001111 & bs[0]) << 12) + \
                               ((0b00111111 & bs[1]) << 6) + \
                                (0b00111111 & bs[2]))
        elif len(bs) == 4:
            code_points.append(((0b00000111 & bs[0]) << 18) + \
                               ((0b00111111 & bs[1]) << 12) + \
                               ((0b00111111 & bs[2]) << 6) + \
                                (0b00111111 & bs[3]))
        else:
            raise UnicodeDecodeError()
    return code_points

More bit twiddling, this time, getting rid of the header bits and combining the bits together. 

The magic numbers on the left-side of `&` are commonly called bit-masks. Recall propositional logic for "and" (`&`):

|`&`|T|F|
|---|-|-|
|T|T|F|
|F|F|F|

Thus, the expression `0b00000111 & x` gets the last three bits of `x` and _masks over_ all the other values with `0`. 

The leading zeroes are not necessary and the following are equivalent:

```python
0b00000111x & x == 0b111 & x == 0x3 & x == 10 & x
```

However, in the above case, we know that we're dealing with bytes, 8-bit values, so it helps to be explicit about which bits we are masking.

In [ ]:
def codepoints_hex(s):
    return list(map(hex, map(ord, s)))

Expanding it for clarity:

```python
list(
    map(hex,
        map(
            ord,
            s
        )
    )
)
```

`map(f, iter)` is a function which takes an iterable (something that you can iterate through, e.g., you can interate through a string's individual characters) and applies the function `f` on the inner element.

First, we take the string `s` and encode each code point into its Unicode numerical value via `ord()`. We then turn that integer in a string of its hexadecimal representation with `hex()`. Lastly, we turn it from a `map` object into a `list`, which is a bit easier to work with.